In [1]:
# AUTHOR: LI WAN | UNIVERSITY OF CAMBRIDGE

import os
import scipy.io as sio
import pandas as pd
import numpy as np
import time

## Input

In [2]:
# parameters
MaxITN = 5000         # max iteration times
Tol = 1e-6            # tolerance       ## 10^(-03) = 1.0000e-03 = 0.001, is this right? 
Status_Mode = 0       # 1: Calibdation mode; 0: Forecast mode
Status_EmpPred = 1    # 1: predict emp-residential location pair; 0: predict residential location only 
Status_HrentPred = 1  # 1: predict house rents; 0: Exogenous house rents
LLCoefIJ = np.array([[0.0,0.0]]) # log-linear transformation coef
D = 250               # number of working days
Lambda = np.array([[1.0,1.0]])

In [3]:
# vairables
LT = len(Lambda[0]) # labour type       ##size(Lambda,2): number of second dimension (variables/columns) - here, the result is 2. 

EmpSeCTot = np.array([[300,1]]) # total employment by socio-economic classification; input if Status_EmpPred == 1
                                # left-side (100) is high-income group; right-hand side (1) is low income group

EmpSeC = np.array([[100,1],   
                   [100,1],
                   [100,1]])    # employment by socio-economic classification; input if Status_EmpPred == 0
                                # from top to bottom - zone 1, 2, 3 respectively. Left (100) is high-income group; right-hand side (1) is low income group
                           
Time1 = np.array([[5,15,30],    # 5 means: time from living in zone 1 and work in zone 1; 15 means: living in zone 1 but work in zone 2
                  [15,5,15],    # the first 15 means: living in zone 2 but work in zone 1
                  [30,15,5]]) 
                                # travel time matrix (Unit: minute)

Time = np.repeat(Time1[None,...],LT,axis=0)   # Time = repmat(Time1, [1,1,LT])    # Time.shape (2,3,3) - this is right. Here, means 2 layers, 3 rows and 3 columns. = size (3,3,2) in Matlab  
Dist = Time # travel distance matrix (Unit: km)


HS = np.array([[1000],
               [1000],
               [1000]])       # housing floorspace - zone 1, 2, 3 respectively         # OR: np.array([1000,1000,1000])

BFS = np.array([[1000],
                [1000],
                [1000]])      # business floorspace

Hrent0 = np.array([[200],
                   [200],
                   [200]])    # unit house rent

Hrent = Hrent0


ZNum = len(HS)      # zone number    # size(HS,1): number of the first dimension (rows) - here, the result is 3

# check and read ZAT.mat file
name_ZAT = 'ZAT'
name_ZAttrI = 'ZAttrI'
name_ZAttrIJ = 'ZAttrIJ'

from functions import read_ZAT
ZAttrI, ZAttrIJ = read_ZAT(LT,ZNum,name_ZAT,name_ZAttrI,name_ZAttrIJ)

------------------- ZAT file exists - Load ZAT file -----------------


In [4]:
# Data input for updating Hrent
Wage = np.array([[10000,10000],   
                 [10000,10000],
                 [10000,10000]])

HSExpShare = np.array([[0.2,0.2],   
                       [0.2,0.2],
                       [0.2,0.2]])


if Status_EmpPred == 1:
    EmpInput = EmpSeCTot
else:
    EmpInput = EmpSeC

## Iteration - Calculate location choice probability

In [5]:
# 0.8737 seconds

start_time = time.time()

from functions import ProbIJ_Mix, Update_Hrent, Calibrate_ZAttr  #import three key functions

if Status_HrentPred == 1:
    print('--------------------------- Iteration starts ------------------------')
    
    for k in list(range(1,MaxITN+1)):
        
        if k == MaxITN:
            print('-------------------------- MaxITN reached --------------------------')
            break
        
        Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum) #add LT,ZNum
        Hrent, Error = Update_Hrent(Output, LT,ZNum,Wage,HSExpShare,Hrent0,HS)
        
        if Error < Tol:
            print('--------------------- Hrent Converged at ITN = {} ------------------'.format(k))
            break
        else:
            Hrent0 = 1.0*Hrent + 0.0*Hrent0
            continue
    
else:
    print('--------------- Calculate location choice probability ---------------')
    Output = ProbIJ_Mix(Status_EmpPred,D,LLCoefIJ,Lambda,EmpInput,Time,Dist,HS,BFS,Hrent0,ZAttrIJ,ZAttrI, LT,ZNum)
    

if Status_Mode == 1:
    print('---------------------- ZATTR Calibration start ----------------------')
    ZAttrIJ,ZAttrI = Calibrate_ZAttr(D,LLCoefIJ,Lambda,Time,HS,BFS,Hrent, LT,ZNum)
    sio.savemat('ZAT(Python).mat', {'ZAttrIJ':ZAttrIJ, 'ZAttrI':ZAttrI})

print("Elapsed time is: %.4f seconds" % (time.time() - start_time)) 

--------------------------- Iteration starts ------------------------
--------------------- Hrent Converged at ITN = 1832 ------------------
Elapsed time is: 0.7856 seconds


## Write output files

In [6]:
from functions import print_outputs
Output_summary = print_outputs (Status_Mode,Status_EmpPred,Status_HrentPred,Output,Hrent,Tol) # run this function
Output_summary.keys()  # display all the table names in the 'output_summary' file

dict_keys(['Metadata', 'MetadataT', 'T_IJ', 'T_IJ_all', 'T_EREW', 'T_Hrents', 'T_JobOppLatCat', 'T_Tran'])

In [7]:
Output_summary['T_EREW'] # select one table to display. (just change this table name to what we're interested in)

ER                    EW          
     Column_A  Column_B    Column_A  Column_B
0  103.672466  0.345575   89.494273  0.298314
1   92.655067  0.308850  121.011454  0.403372
2  103.672466  0.345575   89.494273  0.298314

In [8]:
Output_summary['T_Hrents'] 

,Hrent
0,208.035876
1,185.927687
2,208.035876


In [9]:
Output_summary['Metadata']  # there is a bit of date format issue - I will check it later. 

[['PROJECT NAME: ProbIJ_Model_Test'],
 ['DATE: ', Timestamp('2021-08-19 18:52:32.112003')],
 ['AUTHOR: LI WAN | UNIVERSITY OF CAMBRIDGE'],
 ['PRECISION: ', 1e-06],
 ['MODEL MODE: FORECAST'],
 ['EMPLOTMENT PREDICTION: ENABLED'],
 ['HOUSE RENTS PREDICTION: ENABLED']]

In [10]:
# # check carlibration 
# sio.loadmat('ZAT(Python).mat')['ZAttrI']

# # compare results generated from Matlab
# matZAT = sio.loadmat('Simplified_Matlab_Model_v3_Calibration/ZAT.mat')['ZAT']
# ZAT = matZAT[0,0]    # ZAT.dtype
# ZAttrI = np.moveaxis(ZAT['ZAttrI'], -1, 0)
# ZAttrIJ = np.moveaxis(ZAT['ZAttrIJ'], -1, 0)

# ZAttrI